In [3]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Load and View the data

In [4]:
import pandas as pd

train_data = pd.read_csv('Train_Dataset.csv')
test_data=pd.read_csv('Test_Dataset.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44262 entries, 0 to 44261
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      44262 non-null  object
 1   is_sarcastic  44262 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 691.7+ KB


In [5]:
train_data.head()

,headline,is_sarcastic
0,supreme court votes 7-2 to legalize all worldl...,1
1,hungover man horrified to learn he made dozens...,1
2,emily's list founder: women are the 'problem s...,0
3,send your kids back to school with confidence,0
4,watch: experts talk pesticides and health,0


**Balanced Dataset**

In [6]:
train_data['is_sarcastic'].value_counts()

0    23958
1    20304
Name: is_sarcastic, dtype: int64

In [7]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

train_corpus = normalize_corpus(list(train_data['headline']))
test_corpus = normalize_corpus(list(test_data['headline']))
len(train_corpus)

44262

In [8]:
train_corpus

array(['supreme court votes 72 legalize worldly vices',
       'hungover man horrified learn made dozens plans last night',
       'emilys list founder women problem solvers congress', ...,
       'protesters ejected donald trump rally holding pocket constitutions',
       'italian recipes oldies goodies',
       'area loser blissfully unaffected whims stock market'],
      dtype='<U721')

**Extract TF-IDF Features**

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
train_fit=tf.fit(train_corpus)
train_vec = train_fit.transform(train_corpus)
test_vec= tf.transform(test_corpus)
train_vec.shape

(44262, 115241)

**Linear Regression model**

In [51]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1, random_state=42, solver='liblinear')

In [52]:
y = train_data['is_sarcastic']
lr.fit(train_vec,y) # Fitting Linear regression
predictions = lr.predict(test_vec)
lr.score(train_vec,y)

0.9589941710722516

Random Forest

In [56]:
from sklearn.ensemble import RandomForestClassifier
# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 10) 
 
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(train_vec,y)
 
# performing predictions on the test dataset
y_pred = clf.predict(test_vec)
clf.score(train_vec,y)

0.9965433102887352

**Generation prediction file**

In [57]:
# To create Dataframe of predicted value with particular respective index
res = pd.DataFrame(y_pred) #preditcions are nothing but the final predictions of your model on input features of your new unseen test data
res.index = test_data.index 
res.columns = ["prediction"]

# To download the csv file locally
from google.colab import files
res.to_csv('prediction_results.csv')         
files.download('prediction_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>